In [1]:
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks
from pandapower.tdpf.create_jacobian_tdpf import *

pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", 1000)

In [2]:
net = pp.networks.case30()

In [3]:
for i in net.line.index.values:
    net.line.at[i, 'name'] = f"{net.line.at[i, 'from_bus']+1} - {net.line.at[i, 'to_bus']+1}"

In [4]:
# replicating the calculation from the paper Frank et.al.:
net.line["alpha"] = 1 / (25 + 228.1)
net.line["r_theta"] = calc_r_theta_from_t_rise(net, 25)
net.line["tdpf"] = net.line.r_ohm_per_km != 0

net.line["temperature_degree_celsius"] = 25
net.line["reference_temperature_degree_celsius"] = 25
net.line["ambient_temperature_degree_celsius"] = 25
net.line["outer_diameter_m"] = 30.6e-3
net.line["mc_joule_per_m_k"] = 1490
net.line["wind_speed_m_per_s"] = 0.6
net.line["wind_angle_degree"] = 45
net.line["solar_radiation_w_per_sq_m"] = 900
net.line["gamma"] = 0.5
net.line["epsilon"] = 0.5
# net.line["c_nf_per_km"] = 0  # otherwise i_square_pu will not match net.res_line.i_ka

In [5]:
pp.runpp(net, tdpf=True, tdpf_update_r_theta=False, max_iteration=30)

In [6]:
branch = net._ppc["branch"]
V = net._ppc["internal"]["V"]
Vm = abs(V)
Va = np.angle(V)

In [7]:
# check if p_loss_pu is correct and i_square_pu is correct
g, b = calc_g_b(branch[:, BR_R].real, branch[:, BR_X].real)
i_square_pu, p_loss_pu = calc_i_square_p_loss(branch, np.arange(len(branch)), g, b, Vm, Va)
assert np.allclose(p_loss_pu * net.sn_mva, net.res_line.pl_mw)
# only passes if c_nf_per_km==0
# assert np.allclose(np.sqrt(i_square_pu), net.res_line.i_from_ka / net.sn_mva * (135 * np.sqrt(3)))

In [8]:
v_base_kv = net.bus.loc[net.line.from_bus].vn_kv.values
z_base_ohm = np.square(v_base_kv) / net.sn_mva
net.line["temp"] = net.res_line.temperature_degree_celsius
net.line["loading_percent"] = net.res_line.loading_percent
net.line["pl"] = net.res_line.pl_mw
net.line["r1"] = net.line.r_ohm_per_km / z_base_ohm
net.line["r2"] = net.res_line.r_ohm_per_km / z_base_ohm
net.line["r_delta"] = (net.res_line.r_ohm_per_km - net.line.r_ohm_per_km) / net.line.r_ohm_per_km * 100
net.line["mva"] = np.sqrt(net.res_line.p_from_mw**2 + net.res_line.q_from_mvar**2)
#net.line.sort_values(by="loading_percent", ascending=False)

In [9]:
ref2 = pd.read_csv("case30_branch_details.csv")

In [10]:
# compare p_loss
assert np.allclose(ref2.sort_values(by="R_THETA", ascending=False).PLoss_TDPF * net.sn_mva, 
                   net.line.sort_values(by="r_theta", ascending=False).pl, rtol=0, atol=1e-6)
# compare R_Theta
assert np.allclose(ref2.sort_values(by="R_THETA", ascending=False).R_THETA, 
                   net.line.sort_values(by="r_theta", ascending=False).r_theta)
# compare difference of R
assert np.allclose (ref2.sort_values(by="R_THETA", ascending=False).R_diff.fillna(0) * 100,
                    net.line.sort_values(by="r_theta", ascending=False).r_delta.fillna(0))